# Example 3: Grains Prophet

## Pre-requisities

### 1. Open the example notebook in Colab
Our example notebooks assume you are using Google Colab. To open this notebook in Colab:
1. Sign-up or login to [Google Colab](https://colab.research.google.com/).
1. Select `File`.
1. Select `Open notebook`.
1. Select the `Github` tab.
1. Then:
    1. In `Enter a GitHub URL or search by organisation or user`, enter `SigTechnologies` and select the search option.
    1. In the `Repository` drop-down menu, select `SIGTechnologies/sigtech-python`.
    1. In the `branch` drop-down menu, select `master`.
1. Select the file you want to open.

### 2. Set up your Colab environment

In [ ]:
# Install our Python SDK
!pip install sigtech 

# Import os and our Python SDK
import os
import sigtech.api as sig

# Define your API key as a string. Remember to delete it before sharing your notebook with others. 
os.environ['SIGTECH_API_KEY'] = '<YOUR_API_KEY>'

# Import any additional Python libraries you require.
import numpy as np
import pandas as pd
from tqdm import tqdm
import datetime as dtm

# The prophet library is designed to make time series forecasting tasks more accessible and intuitive.
from prophet import Prophet # The Prophet class is the core component of the library and is used to create a time series forecasting model.
from prophet.plot import plot_plotly, plot_components_plotly # plot_plotly and plot_components_plotly areplotting functions are used to visualize the results of time series forecasting.

### 3. Start a session
After installing our python SDK, defining your API key, importing any Python libraries you require and setting any default parameters, initialize your session.

In [ ]:
sig.init()

## Introduction to a prophet grains strategy

The prophet grains strategy is a trading strategy that leverages the seasonal patterns in the Corn, Soybean, and Wheat futures markets to make trading decisions. The strategy takes advantage of the differences in supply and pricing between the old crop and new crop months. The key components of this strategy are:

- **Seasonal patterns**: the strategy is based on the understanding that the standardized trading months for Corn, Soybean, and Wheat futures align with the seasonal patterns in planting, harvesting, and marketing the underlying crops. For example, spring is the planting season for Corn and Soybeans, while fall is the planting season for Wheat. Similarly, July is the typical harvest month for Wheat, while November and December are the harvest months for Corn and Soybeans.

- **Old crop vs. new crop**: during the planting months, the source of grain available is from crops harvested during the previous harvest season, known as the "old crop." On the other hand, during the harvest months, the newly harvested crop comes to the market, resulting in higher supply, known as the "new crop."

- **Seasonal pricing**: When a new crop is harvested and supply is higher, the grain markets tend to reflect their lowest seasonal prices. Conversely, during the old crop months when supply is typically lower, grain prices tend to be higher compared to the farther out new-crop trading months.

- **Prophet forecasting**: The strategy utilizes the "Prophet" forecasting procedure to make predictions for time series data. Prophet is a model that captures non-linear trends and incorporates yearly, weekly, and daily seasonality, along with holiday effects. It works well with time series data that exhibit strong seasonal patterns and have historical data covering multiple seasons.

### Usefulness of the prophet library

The prophet library is key to this strategy because it is:

- Robust to missing data: prophet is capable of handling missing data points in the time series.
- Handles shifts in the trend: it can identify and adapt to shifts in the underlying trend of the time series.
- Outlier handling: prophet typically performs well even when the data contains outlier values.

Overall, the Prophet Grains Strategy aims to identify favorable trading opportunities by considering the seasonal supply and pricing patterns of Corn, Soybean, and Wheat futures. By using the Prophet forecasting model, the strategy can make informed decisions on when to enter or exit trades based on the historical seasonal trends and pricing behaviors of these agricultural commodities.

## Our strategy


## 1. Forecast the price of one grain

The following code forecasts the price movements of soybean futures using the prophet forecasting model 

In [ ]:
soy = sig.RollingFutureStrategy(contract_code='S', contract_sector='COMDTY')

In [ ]:
# Fetch the historical price data of the Soybean futures using the soy rolling futures strategy.
df_soy = soy.history().reset_index().rename({'date': 'ds', soy.name: 'y'}, axis=1) 

# Initialize a prophet forecasting model. The model will capture yearly and weekly seasonality but not daily patterns.
m_soy = Prophet(daily_seasonality=False) 

# Fit the prophet model to the historical price data of Soybean futures (stored in the df_soy DataFrame).
m_soy.fit(df_soy)  

# Create a DataFrame with future dates to make predictions. Predictions will be made for 365 days into the future.
future_soy = m_soy.make_future_dataframe(periods=365) 

# Filter the future_soy DataFrame to exclude weekends (Saturday and Sunday).
future_soy = future_soy[future_soy['ds'].dt.dayofweek < 5] 

# Use the fitted Prophet model (m_soy) to predict the prices of Soybean futures.
forecast_soy = m_soy.predict(future_soy) 

In [ ]:
# Generates a Plotly interactive plot to visualize the forecasted price values and the historical price data for Soybean futures
plot_plotly(m_soy, forecast_soy)

In [ ]:
plot_components_plotly(m_soy, forecast_soy)

## 3. Build Signal

We will train prophet weekly to predict the next 5 trading days, starting with 3 years of training data.

The provided code builds a signal for the Prophet Grains Strategy by using the train_prophet function to iteratively train the Prophet forecasting model. The goal is to predict the next 5 trading days' price movements for Soybean futures.

Here's an explanation of the code:

train_prophet(df, forecast_days=5): This function trains the Prophet forecasting model (Prophet object) using the historical price data of Soybean futures represented by the DataFrame df. The forecast_days parameter specifies the number of trading days into the future for which the model will make predictions (default is 5 days). The function returns a DataFrame (forecast) containing the predictions for the specified future dates, including the forecasted prices (yhat) and their lower and upper bounds (yhat_lower and yhat_upper).

iter_prophet(df, retrain_after_days=5, forecast_days=5, min_training_days=750): This function iteratively trains the Prophet model using the train_prophet function. The df DataFrame contains historical price data of Soybean futures (or any other commodity). The parameters are as follows:

retrain_after_days: The number of days after which the model will be retrained. This ensures that the model adapts to changing market conditions over time. The default is 5 days.
forecast_days: The number of trading days into the future for which the model will make predictions. This is passed to the train_prophet function.
min_training_days: The minimum number of training days required before making the first prediction. This ensures that the model has sufficient historical data to learn meaningful patterns. The default is 750 days.
iter_forecasts = []: This list will store the forecasts obtained after training the model iteratively.

Iterative Training: The code then iterates through the historical price data in chunks of retrain_after_days. For each iteration, the train_prophet function is called to train the model on the historical data up to the current iteration's end (df.loc[:i+min_training_days]). The forecast for the next forecast_days is obtained from the trained model.

Appending Forecasts: Each forecast obtained in the loop is appended to the iter_forecasts list.

pd.concat(iter_forecasts): Finally, the code concatenates all the forecasts from the iter_forecasts list into a single DataFrame called df_prophet_soy.

In summary, the iter_prophet function utilizes the train_prophet function to train and predict future price movements for Soybean futures iteratively. The iterative training allows the model to be updated regularly with new data, ensuring it captures changing market conditions. The resulting DataFrame df_prophet_soy contains the forecasts for the next 5 trading days based on the Prophet Grains Strategy and can be used to make informed trading decisions.

In [ ]:
def train_prophet(df, forecast_days=5):
    m = Prophet(daily_seasonality=False)
    m.fit(df)
    future = m.make_future_dataframe(periods=forecast_days, include_history=False)
    future = future[future['ds'].dt.dayofweek < 5] # don't predict weekends
    forecast = m.predict(future)
    
    return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

In [ ]:
def iter_prophet(df, retrain_after_days=5, forecast_days=5, min_training_days=750):
    
    iter_forecasts = []
    for i in tqdm(range(0, len(df), retrain_after_days)):
        forecast = train_prophet(df.loc[:i+min_training_days], forecast_days)
        iter_forecasts.append(forecast)
    
    return pd.concat(iter_forecasts)

In [ ]:
df_prophet_soy = iter_prophet(df_soy)

In [ ]:
df_prophet_soy[['yhat', 'yhat_lower', 'yhat_upper']] = df_prophet_soy[['yhat', 'yhat_lower', 'yhat_upper']].shift(-1)
merge_df = df_soy.merge(df_prophet_soy, on='ds', how='outer').dropna().set_index('ds')
merge_df.head()

In [ ]:
merge_df.plot()

Let's compare prophet to a smooth moving average

In [ ]:
merge_df['sma'] = merge_df['y'].rolling(21*3).mean()
merge_df = merge_df.dropna()
merge_df[['y', 'yhat', 'sma']].plot()

We generate a mean reversion signal whenever the price deviates 2 Z scores from our trend line, short when it's above, long when it's below.

In [ ]:
def get_signal(df, mean_reversion_trend='sma', z_threshold = 2):
    daily_pct_diff = (df[mean_reversion_trend] - merge_df['y']) / merge_df['y']
    daily_pct_diff_mean = daily_pct_diff.rolling(63).mean()
    daily_pct_diff_std = daily_pct_diff.rolling(63).std()
    z_score = (daily_pct_diff - daily_pct_diff_mean) / daily_pct_diff_std
    df['signal'] = np.where(z_score < -2, 1, np.where(z_score > 2, -1, 0))
    return df

In [ ]:
df_mean_reversion_prophet_signal = get_signal(merge_df, mean_reversion_trend='yhat')
df_signal = df_mean_reversion_prophet_signal['signal'].rename(soy.name).to_frame()
prophet_strategy = sig.SignalStrategy(
    signal_input=df_signal,
    start_date=df_signal.first_valid_index().date(),
    rebalance_frequency='1BD'
)

In [ ]:
df_mean_reversion_sma_signal = get_signal(merge_df, mean_reversion_trend='sma')
df_signal_sma = df_mean_reversion_sma_signal['signal'].rename(soy.name).to_frame()
sma_strategy = sig.SignalStrategy(
    signal_input=df_signal_sma,
    start_date=df_signal_sma.first_valid_index().date(),
    rebalance_frequency='1BD'
)

In [ ]:
sma_strategy.history().plot(legend=True)
prophet_strategy.history().plot(legend=True)

The prophet strategy outperforms! Let's go ahead and build our basket for three grain contracts - Wheat, Soy, Corn.

## 4. Build the strategy

In [ ]:
def get_prophet_signal(contract_code):
    rfs = sig.RollingFutureStrategy(contract_code=contract_code, contract_sector='COMDTY')
    rfs_df = rfs.history().reset_index().rename({'date': 'ds', rfs.name: 'y'}, axis=1)
    df_prophet = iter_prophet(rfs_df)
    df_prophet[['yhat', 'yhat_lower', 'yhat_upper']] = df_prophet[['yhat', 'yhat_lower', 'yhat_upper']].shift(-1)
    merge_df = rfs_df.merge(df_prophet, on='ds', how='outer').dropna().set_index('ds')
    signal = get_signal(merge_df, mean_reversion_trend='yhat')
    df_signal = signal['signal'].rename(rfs.name).to_frame()
    return df_signal

In [ ]:
all_prophet_signals = []
for c in 'S', 'C', 'W':
    all_prophet_signals.append(get_prophet_signal(c))

In [ ]:
df_all = pd.concat(all_prophet_signals, axis=1) * 0.33

all_prophet_strategy = sig.SignalStrategy(
    signal_input=df_all,
    start_date=dtm.date(2014, 1, 4),
    rebalance_frequency='1BD'
)

In [ ]:
all_prophet_strategy.history().plot()

In [ ]:
df_all.tail()